# CNN + AE


## 1.환경준비

### 1) 라이브러리 로딩

In [ ]:
# import packages
import pandas as pd
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler

import tensorflow.keras as keras
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, Conv1D, Conv1DTranspose
from keras.backend import clear_session

### 2) 필요 함수들 생성

#### ① reconstruction error plot

> * input과 output의 차이(mse)를 계산하고
* 시각화 합니다.



In [ ]:
def recon_err_plot(x, x_pred, y, threshold=0):
    mse = np.mean(np.power(x_val_s.reshape(x.shape[0], -1) - x_pred.reshape(x_pred.shape[0], -1), 2), axis=1)
    error_df = pd.DataFrame({'Reconstruction_error': mse, 'True_class': y})
    error_df = error_df.reset_index()

    groups = error_df.groupby('True_class')
    fig, ax = plt.subplots()
    for name, group in groups:
        ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
                label= "Break" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("Reconstruction error for different classes")
    plt.ylabel("Reconstruction error")
    plt.xlabel("Data point index")
    plt.show()

    return error_df

#### ② precision, recall, f1 curve

> * sklearn에서는 precision, recall curve만 제공됩니다. 
* 그래서, f1 curve도 추가해서 구하고, plot을 그립니다.



In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_rec_f1_curve(y, score, pos = 1) :
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    plt.plot(thresholds, np.delete(precision, -1), label = 'precision')
    plt.plot(thresholds, np.delete(recall, -1), label = 'recall')
    plt.plot(thresholds, np.delete(f1, -1), label = 'f1')
    plt.xlabel('Anomaly Score')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

#### ③ threshold로 잘랐을 때, 분류 평가 함수


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def classification_report2(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    print('< confusion matrix >\n')
    print(confusion_matrix(y, pred_temp))
    print('\n' + '='*60 + '\n')

    print('< classification_report >\n')
    print(classification_report(y, pred_temp))

    return confusion_matrix(y, pred_temp)

#### ④ DL 학습곡선 그리기


In [ ]:
def plot_learning_curve(history) :
    plt.plot(history['loss'], label='Train')
    plt.plot(history['val_loss'], label='Validation')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.show()

#### ⑤ 시계열 데이터 분석을 위한 전처리(for LSTM, CNN)

In [ ]:
def temporalize(x, y, timestep):
    output_X = []
    output_y = []
    for i in range(len(x) - timestep + 1):
        t = []
        for j in range(timestep):
            t.append(x[[(i + j)], :])
        output_X.append(t)
        output_y.append(y[i + timestep - 1])
    return np.squeeze(np.array(output_X)), np.array(output_y)

#### ⑥ 스케일링을 위한 함수
* LSTM+AE를 위한 데이터를 표준화하는 것은 조금 까다로움.
* 원본 데이터는 2D, 모델링 하기 위한 데이터셋(X)은 3D.
* 이를 위해 두 함수를 생성합니다.
    * flatten : temporalize를 원래대로 돌려 놓는 함수
    * scale : 3D 배열의 데이터에 스케일링 적용(transform)

In [ ]:
def flatten(X):
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return flattened_X

def scale(X, scaler):
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])       
    return X

### 3) 데이터셋 불러오기

In [ ]:
# 공정 데이터 불러오기
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/processminer2.csv"
data = pd.read_csv(path)
data.head()

## 2.데이터 준비

### 1) 불필요한 변수 제거

불필요한 변수 제거 : time

In [ ]:
data.drop('time', axis=1, inplace=True)

In [ ]:
data.shape

### 2) 시계열 분석을 위한 데이터 만들기

* x, y 분리

In [ ]:
target = 'y'
input_x = data.drop(target, axis = 1).values
input_y = data['y'].values

In [ ]:
input_x.shape

* timestep에 맞게 data point를 2차원으로 변환  
(data point : 2차원 ==> data set : 3차원)

In [ ]:
timestep = 10 
x, y = temporalize(input_x, input_y, timestep)

In [ ]:
x.shape, y.shape

* 데이터셋 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=6000, random_state=2022)

* Normal 분리 : Normal 데이터로만 학습하기 위해서 

In [ ]:
x_train0 = x_train[y_train==0]

### 3) 스케일링 : Standardization

* 학습은 x_train_0을 사용하므로, 
* 스케일링도 x_train_0 기준으로 수행


In [ ]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(x_train0))
x_train0_s = scale(x_train0, scaler)

In [ ]:
x_val_s = scale(x_val, scaler)

## 3.모델링①



### 1) CNN + AE

In [ ]:
timesteps =  x_train0_s.shape[1] # equal to the lookback
n_features =  x_train0_s.shape[2] # 59

In [ ]:
# Encoder
input_layer = Input(shape=(timesteps, n_features))
encoder = Conv1D(32, 3, activation='relu', padding = 'same')(input_layer)
encoder = Conv1D(16, 3, activation="relu", padding = 'same')(encoder)
# Decoder
decoder = Conv1DTranspose(16, 3, activation="relu", padding = 'same')(encoder)
decoder = Conv1DTranspose(32, 3, activation="relu", padding = 'same')(decoder)
decoder = Conv1DTranspose(n_features, 3, padding = 'same')(decoder)

conv1d_ae = Model(inputs=input_layer, outputs=decoder)

conv1d_ae.summary()

In [ ]:
conv1d_ae.compile(loss='mse', optimizer='adam')

history = conv1d_ae.fit(x_train0_s, x_train0_s
                        , epochs=100, batch_size=128
                        ,validation_split=.2).history

In [ ]:
plot_learning_curve(history)

### 2) 분류에 대한 평가

* test 셋으로 예측하고 reconstruction error로 평가해 봅시다. (recon_err_plot2)

In [ ]:
pred = conv1d_ae.predict(x_val_s)

In [ ]:
result = recon_err_plot(x_val_s, pred, y_val, .2)

* thresholds 값을 조절하면서, precision, recall, f1 score 그래프를 그려 봅시다.(prec_rec_f1_curve)

In [ ]:
precision, recall, f1, thresholds = prec_rec_f1_curve(result['True_class'], result['Reconstruction_error'])

* f1 score를 가장 높이는 thresholds 값을 찾고 평가해 봅시다.

In [ ]:
thres_f1_max = thresholds[np.argmax(f1)]
thres_f1_max

In [ ]:
cm = classification_report2(result['True_class'], result['Reconstruction_error'],thres_f1_max)

## 4.모델링②

### 1) CNN + AE
다음의 구조로 모델링을 수행해 봅시다

| Layer (type) |  Output Shape  |  Param #   
| ---- | ---- | ---- |
|input_3 (InputLayer) |       [(None, 10, 59)]  |        0        | 
| conv1d_2 (Conv1D)   |        (None, 10, 32)   |         5696      |
| conv1d_3 (Conv1D)   |        (None, 10, 16)   |         1552    |  
| conv1d_transpose_3 (Conv1DTranspose) | (None, 10, 16)   |        784      | 
| conv1d_transpose_4 (Conv1DTranspose) | (None, 10, 32)  |        1568      |
| conv1d_transpose_5 (Conv1DTranspose) | (None, 10, 59)   |        5723     | 


### 2) 분류에 대한 평가

* test 셋으로 예측하고 reconstruction error로 평가해 봅시다. (recon_err_plot2)

* thresholds 값을 조절하면서, precision, recall, f1 score 그래프를 그려 봅시다.(prec_rec_f1_curve)

* f1 score를 가장 높이는 thresholds 값을 찾고 평가해 봅시다.

## 5.비즈니스 관점에서의 모델 평가

> * 한 롤로 종이를 말다가 찢어지는 사고가 하루에 한번 정도 발생. 
* 이때마다 공정 중단 및 수율 저하 등, 평균적으로 100백만원의 손실이 발생
* 이를 사전에 감지하는 것은 굉장히 어려움. 이런 사고를 5%만 감소시키더라도 회사 입장에서는 상당한 비용 절감효과 예상.
* 장애가 예상된다면, 속도를 줄여 장애를 예방할 수 있다. 단, 속도를 줄이면 생산성이 저하되므로, 1회당 평균 3만원의 손실이 발생됩니다.


### 1) 비즈니스 기대가치 매트릭스

### 2) cost 계산

* base cost 계산 : 두가지로 계산할 필요가 있습니다.
    * 1) 예방활동을 하지 않고, 장애 발생에 대한 조치 비용 계산(계산가능!)
    * 2) 현재 수행중인 예방활동 비용 + 장애발생 비용 계산(이 부분은 현재 모르므로 여기서는 다루지 않음)

* 예측값에 대한 cost 계산

### 3) 평가해봅시다.